# Non-Standard Phone Insights


In [ ]:
from pathlib import Path
import pandas as pd
import yaml
from IPython.display import display

search_root = Path.cwd()
config_path = None
for candidate in [search_root, *search_root.parents]:
    maybe = candidate / 'config.yaml'
    if maybe.exists():
        config_path = maybe
        project_root = candidate
        break
if config_path is None:
    raise FileNotFoundError("Unable to locate config.yaml from current working directory.")

cfg = yaml.safe_load(config_path.read_text()) or {}
outputs_dir = (project_root / cfg.get('outputs', {}).get('dir', 'output')).resolve()
if not outputs_dir.exists():
    raise FileNotFoundError(f"Outputs directory not found: {outputs_dir}")

contacts_path = outputs_dir / 'consolidated_contacts.csv'
confidence_path = outputs_dir / 'confidence_report.csv'
referral_path = outputs_dir / 'referral_targets.csv'
tagged_path = outputs_dir / 'tagged_contacts.csv'

df = pd.read_csv(contacts_path, dtype=str, keep_default_na=False)
if 'non_standard_phones' not in df.columns:
    raise ValueError("The consolidated contacts file does not contain a 'non_standard_phones' column.")

flagged = df[df['non_standard_phones'].str.strip() != ''].copy()
if flagged.empty:
    print('No non-standard phone numbers recorded in the current dataset.')
else:
    flagged['non_standard_phone_list'] = flagged['non_standard_phones'].str.split('|')
    exploded = flagged.explode('non_standard_phone_list')
    exploded['non_standard_phone_list'] = exploded['non_standard_phone_list'].fillna('').str.strip()
    exploded = exploded[exploded['non_standard_phone_list'] != '']
    display_columns = [
        'contact_id',
        'full_name',
        'company',
        'title',
        'non_standard_phone_list',
        'phones',
    ]
    available = [col for col in display_columns if col in exploded.columns]
    result = exploded[available].sort_values(['contact_id', 'non_standard_phone_list']).head(100)
    display(result)
